In [1]:
#Importing Libraries and Dependency

import json
from nltk.tokenize import sent_tokenize, word_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
from sklearn.preprocessing import Imputer
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn import svm

In [2]:
#Reading raw values form the data set

stemmer = PorterStemmer()
file = '/Users/admin/Documents/Data Exploration/recipe-ingredients-dataset/train.json'
with open(file) as train_file:
    Training_Data = json.load(train_file)

print(Training_Data[0])

{'id': 10259, 'cuisine': 'greek', 'ingredients': ['romaine lettuce', 'black olives', 'grape tomatoes', 'garlic', 'pepper', 'purple onion', 'seasoning', 'garbanzo beans', 'feta cheese crumbles']}


In [3]:
#Ammount of Traing data
len(Training_Data)

39774

In [4]:
Training_Data[0]['ingredients']

['romaine lettuce',
 'black olives',
 'grape tomatoes',
 'garlic',
 'pepper',
 'purple onion',
 'seasoning',
 'garbanzo beans',
 'feta cheese crumbles']

In [5]:
#Creating Arrays and adding ID, Cuisine & Ingridents in different Array in order to analyze them 
cuisine_id = []
cuisine = []
ingredients = []

#Goes through each if tArrayhe object in the test data and appends it to the assoicated 
for i in range(len(Training_Data)):
    cuisine_id.append(Training_Data[i]['id'])
    cuisine.append(Training_Data[i]['cuisine'])
    ingredients.append(Training_Data[i]['ingredients'])
    
df = pd.DataFrame({'id':cuisine_id, 'cuisine':cuisine, 'ingredients':ingredients})
print(df.head(10))

      id      cuisine                                        ingredients
0  10259        greek  [romaine lettuce, black olives, grape tomatoes...
1  25693  southern_us  [plain flour, ground pepper, salt, tomatoes, g...
2  20130     filipino  [eggs, pepper, salt, mayonaise, cooking oil, g...
3  22213       indian                [water, vegetable oil, wheat, salt]
4  13162       indian  [black pepper, shallots, cornflour, cayenne pe...
5   6602     jamaican  [plain flour, sugar, butter, eggs, fresh ginge...
6  42779      spanish  [olive oil, salt, medium shrimp, pepper, garli...
7   3735      italian  [sugar, pistachio nuts, white almond bark, flo...
8  16903      mexican  [olive oil, purple onion, fresh pineapple, por...
9  12734      italian  [chopped tomatoes, fresh basil, garlic, extra-...


In [6]:
#Number of diffent cuisine in the traing set
df['cuisine'].value_counts()

italian         7838
mexican         6438
southern_us     4320
indian          3003
chinese         2673
french          2646
cajun_creole    1546
thai            1539
japanese        1423
greek           1175
spanish          989
korean           830
vietnamese       825
moroccan         821
british          804
filipino         755
irish            667
jamaican         526
russian          489
brazilian        467
Name: cuisine, dtype: int64

In [7]:
new = []
for s in df['ingredients']:
    s = ' '.join(s)
    new.append(s)

df['ing'] = new

In [8]:
l=[]
for s in df['ing']:
    
    #Remove punctuations
    s = re.sub(r'[^\w\s]','',s)
    
    #Remove Digits
    s = re.sub(r"(\d)", "", s)
    
    #Remove content inside paranthesis
    s = re.sub(r'\([^)]*\)', '', s)
    
    #Remove Brand Name
    s = re.sub(u'\w*\u2122', '', s)
    
    #Convert to lowercase
    s = s.lower()
    
print(df.head(10))

      id      cuisine                                        ingredients  \
0  10259        greek  [romaine lettuce, black olives, grape tomatoes...   
1  25693  southern_us  [plain flour, ground pepper, salt, tomatoes, g...   
2  20130     filipino  [eggs, pepper, salt, mayonaise, cooking oil, g...   
3  22213       indian                [water, vegetable oil, wheat, salt]   
4  13162       indian  [black pepper, shallots, cornflour, cayenne pe...   
5   6602     jamaican  [plain flour, sugar, butter, eggs, fresh ginge...   
6  42779      spanish  [olive oil, salt, medium shrimp, pepper, garli...   
7   3735      italian  [sugar, pistachio nuts, white almond bark, flo...   
8  16903      mexican  [olive oil, purple onion, fresh pineapple, por...   
9  12734      italian  [chopped tomatoes, fresh basil, garlic, extra-...   

                                                 ing  
0  romaine lettuce black olives grape tomatoes ga...  
1  plain flour ground pepper salt tomatoes ground... 

In [9]:
vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(df['ing'])

print(X)
#print(vectorizer.get_feature_names())

  (0, 738)	0.3343204746101372
  (0, 522)	0.14568369866765699
  (0, 958)	0.3040361765035925
  (0, 184)	0.20748802168948122
  (0, 1088)	0.38853112215987895
  (0, 2403)	0.23007896012035983
  (0, 1872)	0.1645493089953018
  (0, 2181)	0.23913220198081458
  (0, 1998)	0.10208411357610164
  (0, 1094)	0.10531073154596084
  (0, 2770)	0.15183517837377775
  (0, 1169)	0.35031170238526027
  (0, 1868)	0.26100924108701357
  (0, 247)	0.1398962004921347
  (0, 1527)	0.2663871237012894
  (0, 2295)	0.3426671291173114
  (1, 1860)	0.11553051231205462
  (1, 2877)	0.2107912072389798
  (1, 1707)	0.20955261084571736
  (1, 1666)	0.3984043568116592
  (1, 676)	0.2182707943085128
  (1, 2983)	0.27471923855187746
  (1, 1190)	0.18772612842727063
  (1, 898)	0.19924115060270703
  (1, 2747)	0.2696867216449215
  :	:
  (39772, 676)	0.10613772616897188
  (39772, 2349)	0.052210298712866955
  (39772, 1013)	0.08838193470073283
  (39772, 1094)	0.06122277701590711
  (39773, 490)	0.2569778430076833
  (39773, 2293)	0.400871723348894

In [10]:
len(new)

39774

In [11]:
type(df['ing'][0])

str

In [12]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(df['cuisine'])
df['cuisine']=le.transform(df['cuisine']) 
df['cuisine'].value_counts()

9     7838
13    6438
16    4320
7     3003
3     2673
5     2646
2     1546
18    1539
11    1423
6     1175
17     989
12     830
19     825
14     821
1      804
4      755
8      667
10     526
15     489
0      467
Name: cuisine, dtype: int64

In [13]:
cuisine_map={'0':'brazilian', '1':'british', '2':'cajun_creole', '3':'chinese', '4':'filipino', '5':'french', '6':'greek', '7':'indian', '8':'irish', '9':'italian', '10':'jamaican', '11':'japanese', '12':'korean', '13':'mexican', '14':'moroccan', '15':'russian', '16':'southern_us', '17':'spanish', '18':'thai', '19':'vietnamese'}

In [14]:
Y=[]
Y = df['cuisine']

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size = 0.2, random_state = 100)

In [16]:
# KNeighborsClassifier
for K in range(25):
 K_value = K+1
 neigh = KNeighborsClassifier(n_neighbors = K_value, weights='uniform', algorithm='auto')
 neigh.fit(X_train, y_train) 
 y_pred = neigh.predict(X_test)
 print("Accuracy is ", accuracy_score(y_test,y_pred)*100,"% for K-Value:",K_value)

Accuracy is  69.1137649277184 % for K-Value: 1
Accuracy is  67.68070395977372 % for K-Value: 2
Accuracy is  71.13764927718415 % for K-Value: 3


KeyboardInterrupt: 

In [17]:
#Implement KNN(So we take K value to be 11)
neigh = KNeighborsClassifier(n_neighbors = 11, weights='uniform', algorithm='auto')
neigh.fit(X_train, y_train) 
y_pred = neigh.predict(X_test)
print("Accuracy is ", accuracy_score(y_test,y_pred)*100,"% for K-Value:11")

Accuracy is  73.99120050282842 % for K-Value:11


In [ ]:
df3=pd.DataFrame({'id':y_test.index, 'cuisine':y_test.values})
y_test2=[]
y_test1=df3['cuisine'].tolist()
for i in range(len(df3['cuisine'])):
    y_test2.append(cuisine_map[str(df3['cuisine'][i])])
print(y_test2)

In [ ]:
# OVA SVM Algorithm gives the best accuracy of 78.88% for the given dataset.
lin_clf = svm.LinearSVC(C=1)
lin_clf.fit(X_train, y_train)
y_pred=lin_clf.predict(X_test)
print(accuracy_score(y_test,y_pred)*100)
result=pd.DataFrame({'Actual Cuisine':y_test2, 'Predicted Cuisine':y_test2})
print(result)

In [ ]:

#Random Forest Testing

In [55]:
#Imports
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from subprocess import check_output
import numpy as np
import pandas as pd
from sklearn import pipeline,ensemble,preprocessing,feature_extraction,metrics
from sklearn.model_selection import cross_validate, train_test_split

In [56]:
train=pd.read_json('/Users/admin/Documents/Data Exploration/recipe-ingredients-dataset/train.json')
train.head()

,id,cuisine,ingredients
0,10259,greek,"[romaine lettuce, black olives, grape tomatoes..."
1,25693,southern_us,"[plain flour, ground pepper, salt, tomatoes, g..."
2,20130,filipino,"[eggs, pepper, salt, mayonaise, cooking oil, g..."
3,22213,indian,"[water, vegetable oil, wheat, salt]"
4,13162,indian,"[black pepper, shallots, cornflour, cayenne pe..."


In [57]:
train.ingredients=train.ingredients.apply(' '.join)

In [58]:
train.head()

,id,cuisine,ingredients
0,10259,greek,romaine lettuce black olives grape tomatoes ga...
1,25693,southern_us,plain flour ground pepper salt tomatoes ground...
2,20130,filipino,eggs pepper salt mayonaise cooking oil green c...
3,22213,indian,water vegetable oil wheat salt
4,13162,indian,black pepper shallots cornflour cayenne pepper...


In [59]:
clf=pipeline.Pipeline([
        ('tfidf_vectorizer', feature_extraction.text.TfidfVectorizer(lowercase=True)),
        ('rf_classifier', ensemble.RandomForestClassifier(n_estimators=500,verbose=1,n_jobs=-1))
    ])

In [61]:
# step 1: testing
X_train,X_test,y_train,y_test = train_test_split(train.ingredients,train.cuisine, test_size=0.2)

In [62]:
clf.fit(X_train,y_train)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    3.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   16.7s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   42.3s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:   47.8s finished


Pipeline(memory=None,
         steps=[('tfidf_vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 toke...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                               

In [63]:
y_pred=clf.predict(X_test)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.4s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.8s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.8s finished


In [64]:
metrics.confusion_matrix(y_test,y_pred)

array([[  38,    0,    0,    0,    0,    1,    0,    3,    0,   11,    0,
           0,    0,   12,    0,    0,    7,    0,    4,    0],
       [   1,   39,    0,    1,    0,   24,    1,    2,    3,   19,    0,
           0,    0,    2,    0,    0,   56,    0,    0,    0],
       [   0,    0,  207,    0,    0,    7,    0,    0,    0,   40,    0,
           0,    0,   13,    0,    0,   35,    0,    0,    0],
       [   0,    0,    1,  468,    2,    4,    0,    7,    0,    9,    0,
          10,    4,   10,    0,    0,    8,    0,   10,    2],
       [   3,    0,    1,   34,   73,    1,    0,    1,    0,   10,    0,
           3,    0,    7,    0,    0,   15,    0,    6,    1],
       [   0,    2,    1,    3,    0,  279,    1,    3,    2,  189,    1,
           0,    0,    9,    2,    1,   45,    3,    0,    0],
       [   0,    0,    0,    0,    0,    2,  120,    6,    0,   94,    0,
           0,    1,    4,    0,    2,    2,    0,    0,    0],
       [   0,    1,    0,    0,    0,    

In [65]:
metrics.accuracy_score(y_test,y_pred)

0.7504714016341923

In [68]:
# step 2: real training
test=pd.read_json('/Users/admin/Documents/Data Exploration/recipe-ingredients-dataset/test.json')

In [69]:
test.ingredients = test.ingredients.apply(' '.join)

In [70]:
test.head()

,id,ingredients
0,18009,baking powder eggs all-purpose flour raisins m...
1,28583,sugar egg yolks corn starch cream of tartar ba...
2,41580,sausage links fennel bulb fronds olive oil cub...
3,29752,meat cuts file powder smoked sausage okra shri...
4,35687,ground black pepper salt sausage casings leeks...


In [71]:
clf.fit(train.ingredients,train.cuisine)

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    5.4s
[Parallel(n_jobs=-1)]: Done 184 tasks      | elapsed:   23.0s
[Parallel(n_jobs=-1)]: Done 434 tasks      | elapsed:   56.1s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.1min finished


Pipeline(memory=None,
         steps=[('tfidf_vectorizer',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 toke...
                 RandomForestClassifier(bootstrap=True, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features='auto',
                                        max_leaf_nodes=None,
                               

In [73]:
pred=clf.predict(test.ingredients)

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 184 tasks      | elapsed:    0.2s
[Parallel(n_jobs=8)]: Done 434 tasks      | elapsed:    0.6s
[Parallel(n_jobs=8)]: Done 500 out of 500 | elapsed:    0.7s finished


In [74]:
df=pd.DataFrame({'id':test.id,'cuisine':pred})

In [75]:
df.to_csv('rf_tfidf.csv', columns=['id','cuisine'],index=False)

In [ ]:
# LB score ~ 0.753